# Работа с Excel

Материалы:
* Макрушин С.В. Лекция 7: Работа с Excel
* https://docs.xlwings.org/en/stable/quickstart.html
* https://nbviewer.jupyter.org/github/pybokeh/jupyter_notebooks/blob/master/xlwings/Excel_Formatting.ipynb#search_text


## Задачи для совместного разбора

1. На листе "Рецептура" файла `себестоимостьА_в1.xlsx` для области "Пшеничный хлеб" рассчитать себестоимость всех видов продукции.

2. Результаты расчетов 1.1 сохранить в отдельном столбце области "Пшеничный хлеб"

3. Приблизить форматирование столбца, добавленного в задаче 2 к оформлению всей области.

4. Выполнить 3 с помощью "протягиваемых" формул.

## Лабораторная работа 7.1

1. Загрузите данные из файлов `reviews_sample.csv` (__ЛР2__) и `recipes_sample.csv` (__ЛР5__) в виде `pd.DataFrame`. Обратите внимание на корректное считывание столбца(ов) с индексами. Оставьте в таблице с рецептами следующие столбцы: `id`, `name`, `minutes`, `submitted`, `description`, `n_ingredients`

In [2]:
import csv
import pandas as pd

In [25]:
# загружаем данные из файла reviews_sample.csv
reviews_df = pd.read_csv('reviews_sample.csv', delimiter = ',')

# загружаем данные из файла recipes_sample.csv
recipes_df = pd.read_csv('recipes_sample.csv', delimiter =',', 
                        usecols=['id', 'name', 'minutes', 'submitted', 'description', 'n_ingredients'])

recipes_df

,name,id,minutes,submitted,description,n_ingredients
0,george s at the cove black bean soup,44123,90,2002-10-25,an original recipe created by chef scott meska...,18.0
1,healthy for them yogurt popsicles,67664,10,2003-07-26,my children and their friends ask for my homem...,NaN
2,i can t believe it s spinach,38798,30,2002-08-29,"these were so go, it surprised even me.",8.0
3,italian gut busters,35173,45,2002-07-27,my sister-in-law made these for us at a family...,NaN
4,love is in the air beef fondue sauces,84797,25,2004-02-23,i think a fondue is a very romantic casual din...,NaN
...,...,...,...,...,...,...
29995,zurie s holey rustic olive and cheddar bread,267661,80,2007-11-25,this is based on a french recipe but i changed...,10.0
29996,zwetschgenkuchen bavarian plum cake,386977,240,2009-08-24,"this is a traditional fresh plum cake, thought...",11.0
29997,zwiebelkuchen southwest german onion cake,103312,75,2004-11-03,this is a traditional late summer early fall s...,NaN
29998,zydeco soup,486161,60,2012-08-29,this is a delicious soup that i originally fou...,NaN


    2. Случайным образом выберите 5% строк из каждой таблицы и сохраните две таблицы на разные листы в один файл `recipes.xlsx. Дайте листам названия "Рецепты" и "Отзывы", соответствующие содержанию таблиц. 

In [4]:
pip install xlwings

Note: you may need to restart the kernel to use updated packages.


In [26]:
sample_reviews = reviews_df.sample(frac=0.05)
sample_recipes = recipes_df.sample(frac=0.05)

# создаем файл Excel и сохраняем данные на разные листы
with pd.ExcelWriter('recipes.xlsx', engine='openpyxl') as writer:
    sample_recipes.to_excel(writer, sheet_name='Рецепты')
    sample_reviews.to_excel(writer, sheet_name='Отзывы')

3. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_assign`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи присваивания массива значений диапазону ячеек.

In [6]:
import xlwings as xw

In [7]:
xw.__version__

'0.29.1'

In [27]:
wb = xw.Book('recipes.xlsx') # подключаем книгу
sheet = wb.sheets['Рецепты']

sheet.range('G1').value = 'seconds_assign' # добавляем столбец seconds_assign

time_minutes = sheet.range('D2').expand('down').value # получаем данные из столбца времени выполнения рецепта в минутах

time_seconds = [time * 60 for time in time_minutes] # преобразуем данные в секунды

sheet.range('G2').options(transpose=True).value = time_seconds # присваиваем массив значений диапазону ячеек в столбце E, начиная со строки 2

wb.save() # сохраняем книгу

4. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_formula`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи формул Excel.

In [29]:
wb = xw.Book('recipes.xlsx')
sheet = wb.sheets['Рецепты']

sheet.range('H1').value = 'seconds_formula' # добавляем столбец seconds_formula

# добавляем формулу для вычисления времени выполнения рецепта в секундах
row_count = sheet.range('D2').expand('down').count
for i in range(2, row_count+2):
    sheet.range(f'H{i}').formula = f'=D{i}*60'

5. Сделайте названия всех добавленных столбцов полужирными и выровняйте по центру ячейки.

In [58]:
wb = xw.Book('recipes.xlsx')
sheet = wb.sheets['Рецепты']
sheet.range('G1').api.Font.Bold = True
sheet.range('H1').api.Font.Bold = True
sheet.range('G1').api.HorizontalAlignment = -4108  
sheet.range('H1').api.HorizontalAlignment = -4108 
wb.save()

6. Раскрасьте ячейки столбца `minutes` в соответствии со следующим правилом: если рецепт выполняется быстрее 5 минут, то цвет - зеленый; от 5 до 10 минут - жёлтый; и больше 10 - красный.

In [ ]:
sht1.activate()
a=sht1.range('D2:D1501').value
k=1
for i in a:
    k+=1
    if i<5:
        xw.Range((k, 4)).color = (0, 255, 0)
    elif 5<=i<=10:
        xw.Range((k, 4)).color = (255, 255, 0)
    else:
        xw.Range((k, 4)).color = (255, 0, 0)

7. Добавьте на лист Рецепты столбец n_reviews, содержащий кол-во отзывов для этого рецепта. Выполните задание при помощи формул Excel.

In [ ]:
sht2.range('G1').options(ndim=1).value=['n_reviews']
sht2.range('G2').value = '=COUNTIF($C$2:$C$1501, C2)'
sht2.range('G2:G1501').value = '=COUNTIF($C$2:$C$1501, C2)'

In [ ]:
sht1.range('J1').options(ndim=1).value=['n_reviews']
sht1.range('J2').value = '=vlookup(C2, Отзывы!$C$2:$K$1501, 5, False)'
sht1.range('J2:J1501').value = '=vlookup(C2, Отзывы!$C$2:$K$1501, 5, False)'